In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sickle_Cell_Anemia"
cohort = "GSE84633"

# Input paths
in_trait_dir = "../../input/GEO/Sickle_Cell_Anemia"
in_cohort_dir = "../../input/GEO/Sickle_Cell_Anemia/GSE84633"

# Output paths
out_data_file = "../../output/preprocess/Sickle_Cell_Anemia/GSE84633.csv"
out_gene_data_file = "../../output/preprocess/Sickle_Cell_Anemia/gene_data/GSE84633.csv"
out_clinical_data_file = "../../output/preprocess/Sickle_Cell_Anemia/clinical_data/GSE84633.csv"
json_path = "../../output/preprocess/Sickle_Cell_Anemia/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Callable, Dict, Any, Optional

# 1. Assess gene expression data availability
is_gene_available = True  # Dataset appears to be gene expression data from PBMCs

# 2. Variable availability and conversion functions
# 2.1. Trait availability
trait_row = 2  # From the characteristics dictionary, disease info is at key 2

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (1 for sickle cell disease, 0 for control)
    if 'sickle cell disease' in value.lower():
        return 1
    elif 'control' in value.lower() or 'healthy' in value.lower():
        return 0
    else:
        return None

# 2.2. Age availability
age_row = None  # Age data not available in the sample characteristics

def convert_age(value):
    # This function won't be used since age data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3. Gender availability
gender_row = None  # Gender data not available in the sample characteristics

def convert_gender(value):
    # This function won't be used since gender data is not available
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Save metadata with initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical feature extraction (if trait data is available)
if trait_row is not None:
    try:
        # Extract clinical features - assuming clinical_data variable exists from previous step
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age if age_row is not None else None,
            gender_row=gender_row,
            convert_gender=convert_gender if gender_row is not None else None
        )
        
        # Preview the extracted clinical data
        print("Preview of extracted clinical data:")
        print(preview_df(selected_clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
else:
    print("No trait data available, skipping clinical feature extraction.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers from the previous output
# These look like numeric identifiers (e.g., 2315554, 2315633), not standard human gene symbols
# Standard human gene symbols would be alphanumeric like BRCA1, TP53, etc.
# These numeric IDs are likely probe IDs that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in the gene annotation contain our probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column in gene_annotation contains probe IDs like 2315100, matching what we saw in gene_data
# - 'gene_assignment' column contains gene symbol information

# Define the columns for mapping
probe_id_column = 'ID'
gene_symbol_column = 'gene_assignment'

# 2. Create a mapping dataframe
mapping_df = gene_annotation[[probe_id_column, gene_symbol_column]].copy()
mapping_df = mapping_df.dropna()  # Remove rows with missing gene symbols
mapping_df = mapping_df.rename(columns={gene_symbol_column: 'Gene'}).astype({probe_id_column: 'str'})

# First let's inspect some examples of the gene assignment strings
print("Example gene assignments:")
for i in range(3):
    if i < len(mapping_df) and isinstance(mapping_df.iloc[i]['Gene'], str):
        print(f"Example {i+1}: {mapping_df.iloc[i]['Gene'][:200]}...")

# Apply the extract_human_gene_symbols function to get gene symbols
mapping_df['Gene'] = mapping_df['Gene'].apply(extract_human_gene_symbols)

# Remove rows with empty gene lists
mapping_df = mapping_df[mapping_df['Gene'].apply(len) > 0]

# Preview the mapping dataframe after extraction
print("\nGene mapping preview after extraction:")
print(mapping_df.head(10))
print(f"Total mappings with gene symbols: {len(mapping_df)}")

# 3. Apply gene mapping to convert probe measurements to gene expression
# Convert probe IDs in gene_data to strings to match the mapping
gene_data.index = gene_data.index.astype(str)

# Check overlap between datasets
common_probes = set(gene_data.index) & set(mapping_df['ID'])
print(f"\nNumber of common probes between expression data and mapping: {len(common_probes)}")

# Apply the mapping to get gene expression values
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Preview the mapped gene expression data
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data_mapped.shape}")
if not gene_data_mapped.empty:
    print(f"First few gene symbols: {gene_data_mapped.index[:5]}")
    print(f"Number of genes: {len(gene_data_mapped)}")
else:
    print("Warning: No gene symbols were mapped successfully.")
    
    # If first approach fails, try normalizing the gene symbols
    print("\nAttempting to normalize gene symbols...")
    if len(gene_data_mapped) > 0:
        gene_data_mapped = normalize_gene_symbols_in_index(gene_data_mapped)
        print(f"After normalization - Number of genes: {len(gene_data_mapped)}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_mapped.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Let's try a manual approach to normalize the gene data
# First, extract the gene_data which should be the original probe-level expression data
gene_data_index = gene_data.index.tolist()
print(f"Original gene_data shape: {gene_data.shape}")
print(f"First 10 probes: {gene_data_index[:10]}")

# Check if original gene_data is available and well-formed
if gene_data.shape[0] == 0 or gene_data.shape[1] == 0:
    print("WARNING: Original gene_data is empty. Using original matrix file to get gene data again.")
    # Try to reload gene data from matrix file
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
        # Skip the header line
        next(file)
        # Read the data
        import pandas as pd
        gene_data = pd.read_csv(file, sep='\t', index_col=0)
    print(f"Reloaded gene_data shape: {gene_data.shape}")

# Count the mappable genes
mapping_count = mapping_df.groupby('Gene').size().sort_values(ascending=False)
print(f"Top 10 mapped gene symbols: {mapping_count.head(10)}")

# Try another approach for mapping: keep the original probe IDs if mapping fails
# This means we'll use probe IDs as substitutes for gene symbols
print("\nAttempting to create linked data using probe IDs...")

# 2. Link clinical and expression data
clinical_features = pd.read_csv(out_clinical_data_file)
clinical_features = clinical_features.set_index(clinical_features.columns[0])  # Set first column as index

# Transpose gene_data for linking (so samples are rows)
gene_data_t = gene_data.T
print(f"Transposed gene_data shape: {gene_data_t.shape}")

# Link the data
linked_data = pd.concat([clinical_features.T, gene_data_t], axis=1)
print(f"Linked data shape with probe IDs: {linked_data.shape}")

# Check for missing values
print("\nMissing values summary:")
trait_missing = linked_data[trait].isna().sum()
print(f"  Trait ({trait}) missing: {trait_missing} out of {len(linked_data)}")

# Handle missing values
if linked_data.shape[0] > 0:
    # Get gene columns (all columns except trait, Age, Gender)
    covariate_cols = [trait]
    if 'Age' in linked_data.columns:
        covariate_cols.append('Age')
    if 'Gender' in linked_data.columns:
        covariate_cols.append('Gender')
    
    gene_cols = [col for col in linked_data.columns if col not in covariate_cols]
    
    # Print missing value statistics before cleaning
    gene_missing_pct = linked_data[gene_cols].isna().mean()
    print(f"  Genes with >20% missing: {sum(gene_missing_pct > 0.2)} out of {len(gene_cols)}")
    
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    print(f"  Samples with >5% missing genes: {sum(sample_missing_pct > 0.05)} out of {len(linked_data)}")
    
    # Apply missing value handling
    cleaned_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {cleaned_data.shape}")
    
    # Evaluate bias in trait and demographic features
    if len(cleaned_data) > 0:
        is_trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
        
        # Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains only {trait} patients with no controls. Used probe IDs instead of gene symbols."
        )
        
        # Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be biased or empty and was not saved")
    else:
        print("No data remains after handling missing values.")
        # Record cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=True, 
            df=pd.DataFrame(),
            note=f"Dataset produced empty dataframe after handling missing values."
        )
else:
    print("Linked data is empty.")
    # Record cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=is_gene_available, 
        is_trait_available=is_trait_available, 
        is_biased=True, 
        df=pd.DataFrame(),
        note=f"Failed to create valid linked data with gene expression values."
    )


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note=f"Dataset contains gene expression data but lacks clear trait indicators for {trait} status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note=f"Dataset contains only {trait} patients with no healthy controls, making it unsuitable for case-control analysis."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data for {trait}: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")